# Google Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00


# 4.2 모델 초기화 후 학습

In [1]:
from datasets import load_dataset
from transformers import BertTokenizerFast

dataset = load_dataset("klue", "ynat")
tokenizer = BertTokenizerFast.from_pretrained("drive/MyDrive/Books/outputs/MyTokenizer")

In [2]:
from transformers import BertConfig

cfg = BertConfig
cfg

transformers.models.bert.configuration_bert.BertConfig

In [3]:
mycfg = BertConfig(vocab_size=tokenizer.vocab_size)

In [4]:
from transformers import BertForMaskedLM

model = BertForMaskedLM(mycfg)
model.config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 24000
}

In [11]:
!pip install accelerate -U

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

datasets = dataset.map(
    lambda x: tokenizer(x['title']),
    batched=True,
    batch_size=1000,
    remove_columns=dataset.column_names['train'],
)

args = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    max_steps=1000,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=100,
    logging_dir="drive/MyDrive/Books/outputs/logs",
    output_dir="drive/MyDrive/Books/outputs/ckpt",
)

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
)

trainer.train()

trainer.save("drive/MyDrive/Books/outputs/MyBertModel")